# Describe a PIV Result File

In this example, we want to describe the result file of the ILA Vortex pair (source: https://www.pivtec.com/download/samples/VortexPairSeq.zip)

Where to start? Let's first evaluate what we have:

General context:
- The general concept of a dataset is described by `dcat:Dataset`
- The file is described by `pivmeta:PIVResultDistribution` and is part of the `dcat:Dataset`

Specific information:<br>
Of greater interest is the PIV process including the PIV parameters leading to the dataset. A `dcat:Dataset` is the output of the PIV process. For this, we can use [`m4i:output of`](http://purl.obolibrary.org/obo/RO_0002353).

## Preparation and Imports

In [1]:
from pivmetalib import PIVMETA, pivmeta, dcat, m4i, schema, sd, prov

from ssnolib import StandardNameTable

We will describe variables using standard names. For this we, decide to use [this Standard Name table](https://zenodo.org/records/14175299)

In [2]:
snt = StandardNameTable.download(url="https://zenodo.org/records/14175299/files/Standard_Name_Table_for_Particle_Image_Velociemtry_data.jsonld?download=1", fmt="jsonld")
standard_names = snt.get_standard_names_as_frozen_dataclass()

In [3]:
result_dist = pivmeta.ImageVelocimetryDistribution(
    title='Result File',
    downloadURL='file:///vp1a.dat',
    hasPIVDataType=PIVMETA.ResultData
)
# the "downloaded" file must exist:
assert result_dist.download().exists()

In [4]:
piv_image_dist = pivmeta.ImageVelocimetryDistribution(
    title='ILA Vortex Pair Images',
    download_URL='https://www.pivtec.com/download/samples/pivimg1.zip',
    hasPIVDataType=PIVMETA.ExperimentalImage
)
piv_image_dist

ImageVelocimetryDistribution(id=_:N9f9d1b0b7e1745ff9baa8df83562a7e3, title=ILA Vortex Pair Images, downloadURL=https://www.pivtec.com/download/samples/pivimg1.zip, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)

In [5]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=[result_dist, piv_image_dist]
)
ds

Dataset(id=_:N62d3d10589dd4bbebe5e4da2442c1e0f, title=ILA Vortex Pair, distribution=[ImageVelocimetryDistribution(id=_:N106ddc844be4428e809547a99fd7d676, title=Result File, downloadURL=file:///vp1a.dat, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ResultData), ImageVelocimetryDistribution(id=_:N9f9d1b0b7e1745ff9baa8df83562a7e3, title=ILA Vortex Pair Images, downloadURL=https://www.pivtec.com/download/samples/pivimg1.zip, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)])

Before defining the PIV processing steps, we need to describe the software used:

In [6]:
piv_software = pivmeta.PIVSoftware(
    author=prov.Organization(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)

In [7]:
# import h5py
# from typing import Dict


# import h5rdmtoolbox as h5tbx
# h5tbx.set_config(natural_naming=False)

# with h5tbx.File() as h5:
#     piv_software.dump_hdf(h5)
#     h5.dump()
#
# print(piv_software.dump_jsonld())

## Processing steps

A `PIVProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [13]:
pre = pivmeta.PIVPreProcessing(
    name='Image pre processing',
    realizesMethod=[
        m4i.Method(
            description='Rotates the input image by 180 deg',
            parameter=m4i.NumericalVariable(
                name='rotation',
                hasNumericalValue=180,
                unit='deg',
                quantity_kind='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        m4i.Method(
            description='Applies a median filter to the image',
            hasParameter=m4i.NumericalVariable(
                standard_name=standard_names.image_filter_kernel_size,
                hasNumericalValue=3,
            )
        )
    ]
)
pre

PIVPreProcessing(id=_:N4af066dfd6444073a4bcccbe07789bad, realizesMethod=[Method(id=_:Nd800bf182c3648289366e19627892f45, description=Rotates the input image by 180 deg, parameter=NumericalVariable(id=_:N45b58313183a4a56904958f4ee15d7ff, hasNumericalValue=180.0, name=rotation, unit=deg, quantity_kind=https://qudt.org/vocab/unit/DEG)), Method(id=_:N8e447351e375410f851c45a32c40e758, description=Applies a median filter to the image, hasParameter=NumericalVariable(id=_:N5fb388e6e0c642d1978e0b7515a256b4, hasNumericalValue=3.0, standard_name=image_filter_kernel_size))], name=Image pre processing)

## 2. PIV evaluation

The PIV evaluation is a `m4i:ProcessingStep`. It realizes the methods
- `pivemta:CorrelationMethod` (e.g. FFT, ... ),
- `pivemta:InterrogationMethod` (e.g. `pivemta:Multigrid`, ...) and
- `pivemta:PeakSearchMethod`.

The outlier detection (validation) is a sub-processing step (`pivemta:PIVValidation`), because it can realize multiple methods. Most popular ones are described by classes:
- `pivemta:DynMean`.

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [15]:
calgo = pivmeta.CorrelationMethod(
    name='Standard (FFT) Correlation',
    hasWindowWeightingFunction=PIVMETA.SquareWindow,
    description='The default mode of cross-correlation using FFTs to speed the computation. ' 
    'In principle the sum of pixel-wise multiplication of intensities is computed for each ' 
    'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
print(calgo.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#"
    },
    "@type": "pivmeta:CorrelationMethod",
    "schema:description": "The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007).",
    "pivmeta:hasWindowWeightingFunction": "https://matthiasprobst.github.io/pivmeta#SquareWindow",
    "name": "Standard (FFT) Correlation",
    "@id": "_:N58fe70bc3c654940a119e86156e65c52"
}


### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [18]:
v = m4i.NumericalVariable(
    standard_name=standard_names.x_initial_interrogation_window_size,
    hasNumericalValue=64,
)
print(v.model_dump_jsonld(resolve_keys=False))

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "ssno": "https://matthiasprobst.github.io/ssno#"
    },
    "@type": "m4i:NumericalVariable",
    "m4i:hasNumericalValue": 64.0,
    "@id": "_:N9c570a3dcd834fb687568c4bb70eb03d"
}


In [19]:
#print(v.dump_jsonld())

In [21]:
int_meth = m4i.Method(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    hasParameter=[
        m4i.NumericalVariable(
            standard_name=standard_names.x_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        m4i.NumericalVariable(
            # hasStandardName=PIVMETA.number_of_multigrid_passes,
            description='Number of multigrid passes',
            hasNumericalValue=3,
        )
    ]
)
int_meth.hasParameter[-1]

NumericalVariable(id=_:Nafd0c26f3c8248108098eeca76c54f15, hasNumericalValue=3.0, description=Number of multigrid passes)

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [22]:
median_test = pivmeta.OutlierDetectionMethod(
    name='normalized median test',
    hasParameter=m4i.NumericalVariable(
        label='threshold',
        hasNumericalValue=3.0
    )
)

In [23]:
dyn_mean = pivmeta.OutlierDetectionMethod(
    name='dynamic mean test',
    hasParameter=[
        m4i.NumericalVariable(
            label='mean',
            hasNumericalValue=2.0
        ),
        m4i.NumericalVariable(
            label='var',
            hasNumericalValue=1.0
        )
    ]
)

In [24]:
proc = pivmeta.PIVEvaluation(
    label='piv evaluation',
    realizesMethod=[
        calgo,
        int_meth,
        median_test,
        dyn_mean
    ]
)
proc

PIVEvaluation(id=_:Na90679576628476ea4c49950a949eb26, label=piv evaluation, realizesMethod=[CorrelationMethod(id=_:N58fe70bc3c654940a119e86156e65c52, description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)., hasWindowWeightingFunction=https://matthiasprobst.github.io/pivmeta#SquareWindow, name=Standard (FFT) Correlation), Method(id=_:N5986518f51db4452a63348e24eb12569, description=Run a multigrid PIV algorithm on all images, name=Multigrid interrogation method, hasParameter=[NumericalVariable(id=_:Nf575889ec5fd40fb9ddb004b3c6cd108, hasNumericalValue=64.0, standard_name=x_initial_interrogation_window_size), NumericalVariable(id=_:N3a213a6a09fd475f9a9303357c69cf7f, hasNumericalValue=64.0, standard_name=y_initial_interrogation_window_size), NumericalVariable(id=_:Na4eddcb802304e99b351b6ab38a5c069, hasNumericalValue=16.0, standard_name=x_final_interrogation_window_size), NumericalVariable(id=_:N3356cc2e625e4602bd4e1d5f28967ec8, hasNumericalValue=16.0, standard_name=y_final_interrogation_window_size), NumericalVariable(id=_:Nd3f5c57ef5a04429b554f24f6e2b1bb6, hasNumericalValue=8.0, standard_name=x_final_interrogation_window_overlap_size), NumericalVariable(id=_:Nccebe66a0f6e4031b237a594637d8ce6, hasNumericalValue=8.0, standard_name=y_final_interrogation_window_overlap_size), NumericalVariable(id=_:Nafd0c26f3c8248108098eeca76c54f15, hasNumericalValue=3.0, description=Number of multigrid passes)]), OutlierDetectionMethod(id=_:N99747252e47b438fb50d69707e7ddb8c, name=normalized median test, hasParameter=NumericalVariable(id=_:N4480ec0238be44f5a6877a87baedf128, label=threshold, hasNumericalValue=3.0)), OutlierDetectionMethod(id=_:Nb54ec511434c4b529d98b89226bbb676, name=dynamic mean test, hasParameter=[NumericalVariable(id=_:N7f52ba7f49634effa1f76db2e3532554, label=mean, hasNumericalValue=2.0), NumericalVariable(id=_:Nab666d1001f84e6a987f3da03972dc0f, label=var, hasNumericalValue=1.0)])])

In [27]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    parameter=m4i.NumericalVariable(label='kernel', hasNumericalValue=2.0)
)

In [28]:
post = pivmeta.PIVPostProcessing(
    label='Post processing',
    realizesMethod=data_smoothing
)
post.model_dump_jsonld()         

'{\n    "@context": {\n        "owl": "http://www.w3.org/2002/07/owl#",\n        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",\n        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",\n        "schema": "https://schema.org/",\n        "obo": "http://purl.obolibrary.org/obo/",\n        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",\n        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#",\n        "ssno": "https://matthiasprobst.github.io/ssno#"\n    },\n    "@type": "pivmeta:PIVProcessingStep",\n    "rdfs:label": "Post processing",\n    "m4i:realizesMethod": {\n        "@type": "m4i:Method",\n        "schema:description": "applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.",\n        "m4i:hasParameter": {\n            "@type": "m4i:NumericalVariable",\n            "rdfs:label": "kernel",\n            "m4i:hasNumericalValue": 2.0,\n            "@id": "_:N12630462ef984f1fb4817e1e931a11a2"\n        },

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [29]:
post.hasOutput = ds

## 4. dump PIV run to JSON-LD

In [30]:
piv = m4i.ProcessingStep(
    label='PIV Run',
    startsWith=pre,
    endsWith=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = piv_software
proc.has_employed_tool = piv_software
post.has_employed_tool = piv_software

In [31]:
print(pre.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "obo": "http://purl.obolibrary.org/obo/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#"
    },
    "@type": "pivmeta:PIVPostProcessing",
    "m4i:realizesMethod": [
        {
            "@type": "m4i:Method",
            "schema:description": "Rotates the input image by 180 deg",
            "m4i:hasParameter": {
                "@type": "m4i:NumericalVariable",
                "m4i:hasNumericalValue": 180.0,
                "n

In [32]:
with open('piv_process.json', 'w') as f:
    f.write(proc.model_dump_jsonld())

In [33]:
print(piv.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "obo": "http://purl.obolibrary.org/obo/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#"
    },
    "@type": "m4i:ProcessingStep",
    "rdfs:label": "PIV Run",
    "startsWith": {
        "@type": "pivmeta:PIVPostProcessing",
        "m4i:realizesMethod": [
            {
                "@type": "m4i:Method",
                "schema:description": "Rotates the input image by 180 deg",
                "m4i:hasParameter": {
        